In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)
!nvidia-smi

1.15.2
Fri Jun 19 03:08:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os
from keras.preprocessing.sequence import pad_sequences
import pickle
from torch.utils.data import Dataset, Subset, DataLoader

Using TensorFlow backend.


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Master_Final_Project/Genetic_attack/Code/nlp_adversarial_example_master_pytorch')

In [ ]:
import data_utils
import glove_utils
import models
import display_utils
from goog_lm import LM

In [ ]:
import lm_data_utils
import lm_utils

In [ ]:
#IMDB DATA downloading
%%shell 
wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xvzf aclImdb_v1.tar.gz
rm -f  aclImdb_v1.tar.gz

In [ ]:
#SNLI dataset download
%%shell
wget -O snli_data.zip https://nlp.stanford.edu/projects/snli/snli_1.0.zip 
unzip snli_data -d snli_data
rm -rf snli_data.zip

In [ ]:
#counter-fitted vectors download
%%shell
wget https://raw.githubusercontent.com/nmrksic/counter-fitting/master/word_vectors/counter-fitted-vectors.txt.zip
unzip counter-fitted-vectors.txt.zip
rm counter-fitted-vectors.txt.zip

In [ ]:
%%shell
test -d goog_lm || mkdir goog_lm
cd goog_lm
# Actually download the model files.
# links are from: https://github.com/tensorflow/models/tree/master/research/lm_1b
wget http://download.tensorflow.org/models/LM_LSTM_CNN/graph-2016-09-10.pbtxt
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-base
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-char-embedding
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-lstm
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax0
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax1
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax2
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax3
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax4
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax5
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax6
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax7
wget http://download.tensorflow.org/models/LM_LSTM_CNN/all_shards-2016-09-10/ckpt-softmax8
wget http://download.tensorflow.org/models/LM_LSTM_CNN/vocab-2016-09-10.txt
cd ..

In [ ]:
import build_embeddings

tokenizing...
Dataset built !
Loading Glove Model
Done. 2196007  words loaded!
Number of not found words =  7982
Loading Glove Model
Done. 65713  words loaded!
Number of not found words =  15153
All done


In [ ]:
np.random.seed(1001)
torch.manual_seed(1001)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
VOCAB_SIZE  = 50000
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
doc_len = [len(dataset.test_seqs2[i]) for i in 
           range(len(dataset.test_seqs2))]

In [ ]:
skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)
# [300, 50001] 'UKN':50000; The first has no meaning
MAX_VOCAB_SIZE = 50000
embedding_matrix = np.load('aux_files/embeddings_glove_%d.npy' %(MAX_VOCAB_SIZE))

In [ ]:
import random
from compute_dist import compute_dis
from SA_model import SentimentAnalysis
from data_cluster_seg import Data_infor

### Demonstrating how we find the most similar words

In [ ]:
# Pytorch
for i in range(300, 305):
  nearest, nearest_dist = glove_utils.pick_most_similar_words(compute_dis(i), 20, 0.5)
  print('Closest to `%s` are:' %(dataset.inv_dict[i]))
  for w_id, w_dist in zip(nearest, nearest_dist):
        print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

Closest to `later` are:
 --  subsequent   0.18323109771400037
 --  subsequently   0.1867195991340005
 --  afterward   0.2509214012219998
 --  afterwards   0.2576958961479996
 --  thereafter   0.27419810965900027
 --  trailing   0.33680027128100054
 --  after   0.34520261237799943
 --  then   0.36472839338299945
 --  posterior   0.4310855888390004
 --  following   0.48330736760400006
Closest to `takes` are:
 --  pick   0.311305465632
 --  taking   0.4247115846279994
 --  picked   0.4852741249590007
Closest to `instead` are:
 --  conversely   0.3034038049850005
 --  however   0.34753828658300034
 --  alternatively   0.39540487543000014
 --  alternately   0.4439627395600001
 --  nevertheless   0.4771639757920001
Closest to `seem` are:
 --  seems   0.007052995653001437
 --  appears   0.328372447352
 --  looks   0.3353463830640009
 --  transpires   0.456207185493001
Closest to `beautiful` are:
 --  gorgeous   0.019236443661999614
 --  wonderful   0.10149643378300022
 --  splendid   0.102990

## Loading the Google Language model

In [ ]:
goog_lm = LM()


LM vocab loading done

Instructions for updating:
Use tf.gfile.GFile.



Recovering graph.



INFO:tensorflow:Recovering Graph goog_lm/graph-2016-09-10.pbtxt


Recovering checkpoint goog_lm/ckpt-*


### Preparing the dataset

In [ ]:
max_len = 250
train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
train_y = np.array(dataset.train_y)
test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
test_y = np.array(dataset.test_y)

In [ ]:
# pytorch

max_len = 250
padded_train_raw = pad_sequences(dataset.train_seqs2, maxlen = max_len, padding = 'post')
padded_test_raw = pad_sequences(dataset.test_seqs2, maxlen = max_len, padding = 'post')
# TrainSet
data_set = Data_infor(padded_train_raw, dataset.train_y)
num_train = len(data_set)
indx = list(range(num_train))
train_set = Subset(data_set, indx)

# TestSet
SAMPLE_SIZE = 5000
data_set = Data_infor(padded_test_raw, dataset.test_y)
num_test = len(data_set)
indx = list(range(num_test))
# indx = random.sample(indx, SAMPLE_SIZE)
test_set = Subset(data_set, indx)

In [ ]:
save_path = '/content/drive/My Drive/Master_Final_Project/Genetic_attack/Code/nlp_adversarial_example_master_pytorch'

### Loading the sentiment analysis model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_matrix = torch.tensor(embedding_matrix.T).to(device)

In [ ]:
rnn_state_save = os.path.join(save_path,'best_SA')
batch_size = 1
lstm_size = 128

model = SentimentAnalysis(batch_size=batch_size, embedding_matrix = embedding_matrix, hidden_size = lstm_size, kept_prob = 0.9)
model.eval()
model.load_state_dict(torch.load(rnn_state_save))
model.to(device)

SentimentAnalysis(
  (embed): Embedding(50001, 300)
  (lstm): LSTM(300, 128, num_layers=2)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
  (dropout): Dropout(p=0.09999999999999998, inplace=False)
)

In [ ]:
test_loader = DataLoader(test_set,batch_size = int(num_test/10), shuffle = False, pin_memory=True)
model.eval()
test_pred = torch.tensor([])
test_targets = torch.tensor([])
test_loss = []

with torch.no_grad():
  for batch_index, (seqs, length, target) in enumerate(test_loader):
    seqs, target, length = seqs.to(device), target.to(device), length.to(device)
    seqs = seqs.type(torch.LongTensor)
    len_order = torch.argsort(length, descending = True)
    length = length[len_order]
    seqs = seqs[len_order]
    target = target[len_order]

    output = model.pred(seqs, length)
    test_pred = torch.cat((test_pred, output.cpu()), dim = 0)
    test_targets = torch.cat((test_targets, target.type(torch.float).cpu()))

  accuracy = model.evaluate_accuracy(test_pred.numpy(), test_targets.numpy())
  print('Test Accuracy:{:.4f}.'.format(accuracy))

Test Accuracy:0.9011.


In [ ]:
# TestSet
SAMPLE_SIZE = 5000
data_set = Data_infor(padded_test_raw, dataset.test_y)
num_test = len(data_set)
indx = list(range(num_test))
indx = random.sample(indx, SAMPLE_SIZE)
test_set = Subset(data_set, indx)
test_loader = DataLoader(test_set,batch_size = 1, shuffle = False, pin_memory=True)

In [ ]:
torch.save(test_loader, os.path.join(save_path, 'test_loader'))

#### demonstrating the GoogLM

In [ ]:
src_word = dataset.dict['play']
nearest, nearest_dist = glove_utils.pick_most_similar_words(compute_dis(src_word),20)
nearest_w = [dataset.inv_dict[x] for x in nearest]
print('Closest to `%s` are %s' %(dataset.inv_dict[src_word], nearest_w))

Closest to `play` are ['playing', 'gaming', 'games', 'toy', 'playback', 'game', 'plaything', 'cheek', 'gambling', 'toys', 'toying', 'replay', 'stake', 'plays', 'jeu', 'gamble', 'staking', 'reproduction', 'casino', 'sets']


In [ ]:
prefix = 'is'
suffix = 'with'
lm_preds = goog_lm.get_words_probs(prefix, nearest_w, suffix)
print('most probable is ', nearest_w[np.argmax(lm_preds)])


most probable is  game


## Try Attack

In [ ]:
from attacks import GeneticAtack

## Main Attack 

In [ ]:
n1 = 8
n2 = 4
pop_size = 60
max_iters = 30

batch_model = SentimentAnalysis(batch_size=pop_size, embedding_matrix = embedding_matrix, hidden_size = lstm_size, kept_prob = None)

batch_model.eval()
batch_model.load_state_dict(torch.load(rnn_state_save))
batch_model.to(device)

neighbour_model = SentimentAnalysis(batch_size=batch_size, embedding_matrix = embedding_matrix, hidden_size = lstm_size, kept_prob = None)

neighbour_model.eval()
neighbour_model.load_state_dict(torch.load(rnn_state_save))
neighbour_model.to(device)


In [ ]:

class GeneticAttack_pytorch(object):
  def __init__(self, model, batch_model, neighbour_model, compute_dis,
               goog_lm, max_iters, dataset,
               pop_size, n1, n2,
               use_lm = True, use_suffix = False):
    self.model = model
    self.batch_model = batch_model
    self.neighbour_model = neighbour_model
    self.model.eval()
    self.batch_model.eval()
    self.neighbour_model.eval()
    self.compute_dist = compute_dis
    self.pop_size = pop_size
    self.top_n1 = n1
    self.top_n2 = n2
    self.use_lm = use_lm
    self.use_suffix = use_suffix
    self.w_i_dict = dataset.dict
    self.i_w_dict = dataset.inv_dict
    self.dataset = dataset
    self.lm = goog_lm
    self.temp = 0.3
    self.max_iters = max_iters
    

  def attack(self, seq, target, l, max_change = 0.4):
    seq = seq.numpy().squeeze()
    seq_adv = seq.copy()
    seq_len = np.sum(np.sign(seq))
    l = l.cpu()
    # To calculate the sampling probability 
    tmp = [glove_utils.pick_most_similar_words(self.compute_dist(i), ret_count = 50, threshold = 0.5) for i in seq]
    neighbour_list = [t[0] for t in tmp]
    neighbour_dist = [t[1] for t in tmp]
    neighbour_len = [len(i) for i in neighbour_list]
    for i in range(seq_len):
      if seq[i] < 27:
        neighbour_len[i] = 0
    prob_select = neighbour_len/np.sum(neighbour_len)
    tmp = [glove_utils.pick_most_similar_words(
        self.compute_dist(i), self.top_n1, 0.5
    ) for i in seq]
    neighbour_list = [t[0] for t in tmp]
    neighbour_dist = [t[1] for t in tmp]
    pop = [self.perturb(seq_adv, seq, neighbour_list, neighbour_dist, prob_select, seq_len, target, l) for _ in range(self.pop_size)]

    l_tensor = l*torch.ones([len(pop)])
    pop_np = np.expand_dims(pop[0], 0)
    for p in pop[1:]:
      pop_np = np.concatenate((pop_np, np.expand_dims(p, 0)),0) 

    for i in range(self.max_iters):
      pop_tensor = torch.tensor(pop_np).type(torch.LongTensor).to(device)
      l_tensor = l_tensor.to(device)
      self.batch_model.eval()
      with torch.no_grad():
        pop_preds = self.batch_model.pred(pop_tensor, l_tensor).cpu().detach().numpy()
      
      pop_scores = pop_preds[:, target]
      print('\t\t', i, ' -- ', np.max(pop_scores))
      pop_ranks = np.argsort(pop_scores)[::-1]
      top_attack = pop_ranks[0]

      logits = np.exp(pop_scores/self.temp)
      select_probs = logits/np.sum(logits)
    
      if np.argmax(pop_preds[top_attack, :]) == target:
        print('Success and score: {:.4f}'.format(pop_scores[top_attack]))
        return pop[top_attack]
      
      elite = [pop[top_attack]]  # elite
      # print(select_probs.shape)
      parent1_idx = np.random.choice(
          self.pop_size, size=self.pop_size-1, p=select_probs)
      parent2_idx = np.random.choice(
          self.pop_size, size=self.pop_size-1, p=select_probs)
      
      childs = [self.crossover(pop[parent1_idx[i]],
                                pop[parent2_idx[i]])
                for i in range(self.pop_size-1)]
      childs = [self.perturb(
          x, seq, neighbour_list, neighbour_dist, prob_select, seq_len, target, l) for x in childs]
      pop = elite + childs
      pop_np = np.expand_dims(pop[0], 0)
      for p in pop[1:]:
        pop_np = np.concatenate((pop_np, np.expand_dims(p, 0)),0)

    return None
    
  def perturb(self, seq_cur, seq, neighbour_list, neighbour_dist, prob_select, seq_len, target, l):
    prob_len = len(prob_select)
    rand_idx = np.random.choice(prob_len, 1, p = prob_select)[0]
    while seq_cur[rand_idx] != seq[rand_idx] and np.sum(seq != seq_cur)<np.sum(np.sign(prob_select)):
      rand_idx = np.random.choice(prob_len, 1, p = prob_select)[0]
    
    replace_list = neighbour_list[rand_idx]
    if len(replace_list) < self.top_n1:
      replace_list = np.concatenate((replace_list, np.zeros(self.top_n1 - replace_list.shape[0])))
    return self.select_best_replacement(rand_idx, seq_cur, seq, target, l, replace_list)
  
  def replace(self, seq_cur, loc, w):
    seq_new = seq_cur.copy()
    seq_new[loc] = w
    return seq_new


  def select_best_replacement(self, loc, seq_cur, seq, target, l, replace_list):
    new_seq_list = [self.replace(seq_cur, loc, w) if seq[loc]!=w and w != 0 else seq_cur for w in replace_list]
    l_seq_list = len(new_seq_list)
    new_seq_list_tensor = torch.tensor(new_seq_list).type(torch.LongTensor).to(device)
    l_tensor = l*torch.ones([l_seq_list])
    l_tensor = l_tensor.to(device)
    self.neighbour_model.eval()
    with torch.no_grad():
      new_seq_preds = self.neighbour_model.pred(new_seq_list_tensor, l_tensor).cpu().detach().numpy()
    new_seq_scores = new_seq_preds[:, target]
    seq_tensor = torch.tensor(np.expand_dims(seq, axis = 0)).type(torch.LongTensor).to(device)
    l_tensor = l.to(device)
    self.model.eval()
    with torch.no_grad():
      orig_score = self.model.pred(seq_tensor, l_tensor).cpu().detach().numpy()[0, target]
    new_seq_scores -= orig_score
    
    new_seq_scores[self.top_n1:] = -10000000
    
    if self.use_lm:
      prefix = ''
      suffix = None
      if loc > 0:
        prefix = self.i_w_dict[seq_cur[loc-1]]


      orig_word = self.i_w_dict[seq[loc]]
      if self.use_suffix and loc < seq_cur.shape[0]-1 and seq_cur[pos+1]!=0:
        suffix = self.i_w_dict[seq_cur[pos+1]]
      replace_words_orig = [self.dataset.inv_dict[w] if w in self.dataset.inv_dict else 'UNK' for w in replace_list[:self.top_n1]] + [orig_word]

      replace_words_scores = self.lm.get_words_probs(prefix, replace_words_orig, suffix)
        
      new_words_scores = np.array(replace_words_scores[:-1])
      rank_replaces_by_lm = np.argsort(-new_words_scores)
      filtered_words_idx = rank_replaces_by_lm[self.top_n2:]

      new_seq_scores[filtered_words_idx] = -10000000

    if np.max(new_seq_scores)>0:    
      return new_seq_list[np.argsort(new_seq_scores)[-1]]
    return seq_cur

  def crossover(self, seq1, seq2):
    seq_new = seq1.copy()
    for i in range(len(seq1)):
        if np.random.uniform() < 0.5:
            seq_new[i] = seq2[i]
    return seq_new

In [ ]:
ga_attack = GeneticAttack_pytorch(model, batch_model, neighbour_model, compute_dis,
               goog_lm, max_iters = max_iters, dataset = dataset,
               pop_size = pop_size, n1 = n1, n2 = n2,
               use_lm = True, use_suffix = False)

In [ ]:
TEST_SIZE = 200
order_pre = 0
n = 0
seq_success = []
seq_orig = []
seq_orig_label = []
word_varied = []
test_loader = torch.load(os.path.join(save_path, 'test_loader'))
if order_pre != 0:
  seq_success = np.load(os.path.join(save_path,'seq_success.npy'))
  seq_orig = np.load(os.path.join(save_path,'seq_orig.npy'))
  seq_orig_label = np.load(os.path.join(save_path,'seq_orig_label.npy'))
  word_varied = np.load(os.path.join(save_path,'word_varied.npy'))
  n = len(seq_success)

for order, (seq, l, target) in enumerate(test_loader):

  if order>=order_pre:
    print('Sequence number:{}'.format(order))
    seq_len = np.sum(np.sign(seq.numpy()))
    seq, l = seq.to(device), l.to(device)
    seq = seq.type(torch.LongTensor)
    model.eval()
    with torch.no_grad():
      orig_pred = np.argmax(model.pred(seq, l).cpu().detach().numpy())
    if orig_pred != target.numpy()[0]:
      print('Wrong original prediction')
      print('----------------------')
      continue
    if seq_len > 100:
      print('Sequence is too long')
      print('----------------------')
      continue

    print('Length of sentence: {}, Number of samples:{}'.format(l.item(), n+1))
    seq_orig.append(seq[0].numpy())
    seq_orig_label.append(target.numpy()[0])
    target = 1-target.numpy()[0]
    seq_success.append(ga_attack.attack(seq, target, l))
    
    if len(seq_success[n]) > 1:
      w_be = [dataset.inv_dict[seq_orig[n][i]] for i in list(np.where(seq_success[n] != seq_orig[n])[0])]
      w_to = [dataset.inv_dict[seq_success[n][i]] for i in list(np.where(seq_success[n] != seq_orig[n])[0])]
      for i in range(len(w_be)):
        print('{} ----> {}'.format(w_be[i], w_to[i]))
      word_varied.append([w_be]+[w_to])
    else:
      print('Fail')
    print('----------------------')
    n += 1
    
    np.save(os.path.join(save_path,'seq_success.npy'), np.array(seq_success))
    np.save(os.path.join(save_path,'seq_orig.npy'), np.array(seq_orig))
    np.save(os.path.join(save_path,'seq_orig_label.npy'), np.array(seq_orig_label))
    np.save(os.path.join(save_path,'word_varied.npy'), np.array(word_varied))
    
    if n>TEST_SIZE:
      break 

Sequence number:0
Sequence is too long
----------------------
Sequence number:1
Sequence is too long
----------------------
Sequence number:2
Sequence is too long
----------------------
Sequence number:3
Sequence is too long
----------------------
Sequence number:4
Sequence is too long
----------------------
Sequence number:5
Sequence is too long
----------------------
Sequence number:6
Sequence is too long
----------------------
Sequence number:7
Sequence is too long
----------------------
Sequence number:8
Sequence is too long
----------------------
Sequence number:9
Length of sentence: 62, Number of samples:1
		 0  --  1.2732394e-25
		 1  --  2.358753e-19
		 2  --  4.347459e-18
		 3  --  2.0122895e-13
		 4  --  2.7647e-10
		 5  --  7.271452e-10
		 6  --  4.1930864e-08
		 7  --  0.000110052315
		 8  --  0.0016444275
		 9  --  0.9882933
Success and score: 0.9883
trio ----> triangular
related ----> connected
themes ----> items
all ----> everyone
quite ----> rather
executed ----> conduc

In [ ]:
SAMPLE_SIZE = 5000
TEST_SIZE = 200
test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
test_len = []
for i in range(SAMPLE_SIZE):
    test_len.append(len(dataset.test_seqs2[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(SAMPLE_SIZE):
    x_orig = test_x[test_idx[i]]
    orig_label = test_y[test_idx[i]]
    orig_preds=  model.predict(sess, x_orig[np.newaxis, :])[0]
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        #print('skipping wrong classifed ..')
        #print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig))
    if x_len >= 100:
        #print('skipping too long input..')
        #print('--------------------------')
        continue
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    print('****** ', len(test_list) + 1, ' ********')
    test_list.append(test_idx[i])
    orig_list.append(x_orig)
    target_label = 1 if orig_label == 0 else 0
    orig_label_list.append(orig_label)
    x_adv = ga_atttack.attack( x_orig, target_label)
    adv_list.append(x_adv)
    if x_adv is None:
        print('%d failed' %(i+1))
        dist_list.append(100000)
    else:
        num_changes = np.sum(x_orig != x_adv)
        print('%d - %d changed.' %(i+1, num_changes))
        dist_list.append(num_changes)
        # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
    print('--------------------------')
    if (len(test_list)>= TEST_SIZE):
        break

Shortest sentence in our test set is 18 words
******  1  ********
		 0  --  0.16752617
		 1  --  0.33954847
		 2  --  0.62575793
1 - 4 changed.
--------------------------
******  2  ********
		 0  --  0.081095785
		 1  --  0.23031871
		 2  --  0.49455175
		 3  --  0.8090076
3 - 6 changed.
--------------------------
******  3  ********
		 0  --  0.20659587
		 1  --  0.29161933
		 2  --  0.34426233
		 3  --  0.58601284
7 - 5 changed.
--------------------------
******  4  ********
		 0  --  0.00016315776
		 1  --  0.00029146814
		 2  --  0.00045838807
		 3  --  0.0006008647
		 4  --  0.0008673914
		 5  --  0.001375048
		 6  --  0.0021017992
		 7  --  0.0031949652
		 8  --  0.004548417
		 9  --  0.006179247
		 10  --  0.008785875
		 11  --  0.02230807
		 12  --  0.10363737
		 13  --  0.10363737
		 14  --  0.10363737
		 15  --  0.10363737
		 16  --  0.10363737
		 17  --  0.2133652
		 18  --  0.3666688
		 19  --  0.499423
		 20  --  0.8521438
18 - 25 changed.
--------------------------
*****

		 0  --  0.017261371
		 1  --  0.02037716
		 2  --  0.023573171
		 3  --  0.08914166
		 4  --  0.53655946
517 - 6 changed.
--------------------------
******  49  ********
		 0  --  0.8403348
529 - 1 changed.
--------------------------
******  50  ********
		 0  --  0.029262105
		 1  --  0.24247132
		 2  --  0.24247132
		 3  --  0.42393875
		 4  --  0.60402477
531 - 6 changed.
--------------------------
******  51  ********
		 0  --  0.5046299
533 - 1 changed.
--------------------------
******  52  ********
		 0  --  0.42939952
		 1  --  0.939006
540 - 3 changed.
--------------------------
******  53  ********
		 0  --  0.6096896
541 - 1 changed.
--------------------------
******  54  ********
		 0  --  0.4184872
		 1  --  0.5380146
545 - 3 changed.
--------------------------
******  55  ********
		 0  --  0.00039928843
		 1  --  0.0005260108
		 2  --  0.002020025
		 3  --  0.002020025
		 4  --  0.0048475764
		 5  --  0.0066693197
		 6  --  0.013560566
		 7  --  0.015769396
		 8  --  0

******  93  ********
		 0  --  0.74539155
1040 - 1 changed.
--------------------------
******  94  ********
		 0  --  0.0011191729
		 1  --  0.012057994
		 2  --  0.08946816
		 3  --  0.45718634
		 4  --  0.8232168
1045 - 5 changed.
--------------------------
******  95  ********
		 0  --  0.023614038
		 1  --  0.40257096
		 2  --  0.49651057
		 3  --  0.98317194
1057 - 5 changed.
--------------------------
******  96  ********
		 0  --  0.023407836
		 1  --  0.57342935
1066 - 1 changed.
--------------------------
******  97  ********
		 0  --  4.483463e-06
		 1  --  5.2084292e-06
		 2  --  1.7096958e-05
		 3  --  1.7096958e-05
		 4  --  2.7275075e-05
		 5  --  0.00013349178
		 6  --  0.00013349178
		 7  --  0.00013349178
		 8  --  0.016644001
		 9  --  0.03548153
		 10  --  0.03548153
		 11  --  0.03548153
		 12  --  0.13038312
		 13  --  0.13038312
		 14  --  0.13038312
		 15  --  0.13038312
		 16  --  0.13038312
		 17  --  0.29166052
		 18  --  0.30049193
		 19  --  0.88937527
1085 

		 16  --  0.27449238
		 17  --  0.46243197
		 18  --  0.5401243
1394 - 24 changed.
--------------------------
******  138  ********
		 0  --  0.01837934
		 1  --  0.032186992
		 2  --  0.070028216
		 3  --  0.070028216
		 4  --  0.096304245
		 5  --  0.19163771
		 6  --  0.20079419
		 7  --  0.25623515
		 8  --  0.40685016
		 9  --  0.5189224
1405 - 13 changed.
--------------------------
******  139  ********
		 0  --  0.0008616909
		 1  --  0.0025734382
		 2  --  0.094167694
		 3  --  0.21284099
		 4  --  0.43182877
		 5  --  0.7427157
1423 - 8 changed.
--------------------------
******  140  ********
		 0  --  0.103433155
		 1  --  0.56482273
1426 - 1 changed.
--------------------------
******  141  ********
		 0  --  0.012065027
		 1  --  0.15197961
		 2  --  0.8119772
1434 - 5 changed.
--------------------------
******  142  ********
		 0  --  0.013773772
		 1  --  0.018129138
		 2  --  0.09367223
		 3  --  0.29025075
		 4  --  0.33429804
		 5  --  0.34540194
		 6  --  0.38546276


******  191  ********
		 0  --  0.002563564
		 1  --  0.0055873785
		 2  --  0.0059753605
		 3  --  0.007867068
		 4  --  0.013503993
		 5  --  0.0135335745
		 6  --  0.018860644
		 7  --  0.02545974
		 8  --  0.03287588
		 9  --  0.03287588
		 10  --  0.046571806
		 11  --  0.070145756
		 12  --  0.08857982
		 13  --  0.08857982
		 14  --  0.10529671
		 15  --  0.1344016
		 16  --  0.16498345
		 17  --  0.16498345
		 18  --  0.21268459
		 19  --  0.21677099
		 20  --  0.21677099
		 21  --  0.22076964
		 22  --  0.23354492
		 23  --  0.2546097
		 24  --  0.29033723
		 25  --  0.29033723
		 26  --  0.31929314
		 27  --  0.31929314
		 28  --  0.71250224
1873 - 19 changed.
--------------------------
******  192  ********
		 0  --  0.59134144
1874 - 1 changed.
--------------------------
******  193  ********
		 0  --  0.41071123
		 1  --  0.8056726
1880 - 2 changed.
--------------------------
******  194  ********
		 0  --  0.464266
		 1  --  0.8201281
1886 - 2 changed.
-------------------

## Compute Attack success rate

In [ ]:
orig_len = [np.sum(np.sign(x)) for x in orig_list]
normalized_dist_list = [dist_list[i]/orig_len[i] for i in range(len(orig_list)) ]

In [ ]:
SUCCESS_THRESHOLD  = 0.25
successful_attacks = [x < SUCCESS_THRESHOLD for x in normalized_dist_list]
print('Attack success rate : {:.2f}%'.format(np.mean(successful_attacks)*100))
print('Median percentange of modifications: {:.02f}% '.format(
    np.median([x for x in normalized_dist_list if x < 1])*100))
print('Mean percentange of modifications: {:.02f}% '.format(
    np.mean([x for x in normalized_dist_list if x < 1])*100))

Attack success rate : 92.00%
Median percentange of modifications: 6.45% 
Mean percentange of modifications: 9.14% 


In [ ]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

Original Prediction = Negative. (Confidence = 99.81) 


---------  After attack -------------
New Prediction = Positive. (Confidence = 59.80) 


In [ ]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

Original Prediction = Positive. (Confidence = 85.73) 


---------  After attack -------------
New Prediction = Negative. (Confidence = 50.19) 


In [ ]:
## Save success
with open('attack_results_final.pkl', 'wb') as f:
    pickle.dump((test_list, orig_list, orig_label_list, adv_list, normalized_dist_list), f)
    
